In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import re

from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer


In [21]:
with open('krovostok.txt', 'r', encoding='utf-8') as f: # загружается текстовый файл
    text = f.read()
    text = text.replace('\ufeff', '')  # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text)  # заменяем все символы кроме кириллицы на пустые символы
    
# парсим текст, как последовательность символов
num_characters = 34  # 33 буквы + пробел
tokenizer = Tokenizer(num_words=num_characters, char_level=True)  # токенизируем на уровне символов
tokenizer.fit_on_texts([text])  # формируем токены на основе частотности в нашем тексте
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'а': 3, 'е': 4, 'и': 5, 'т': 6, 'н': 7, 'л': 8, 'с': 9, 'к': 10, 'р': 11, 'в': 12, 'у': 13, 'м': 14, 'д': 15, 'п': 16, 'б': 17, 'ы': 18, 'я': 19, 'з': 20, 'ь': 21, 'ч': 22, 'г': 23, 'х': 24, 'й': 25, 'ш': 26, 'ж': 27, 'ю': 28, 'ц': 29, 'э': 30, 'щ': 31, 'ф': 32, 'ъ': 33}


In [22]:
inp_chars = 5 # прогноз на основе 6 -> берем 6 символов из коллекции, а прогнозируем уже 7й 
data = tokenizer.texts_to_matrix(text)  # преобразуем исходный текст в массив OHE
n = data.shape[0] - inp_chars  # так как мы предсказываем по шести символам - седьмой

X = np.array([data[i:i + inp_chars, :] for i in range(n)]) # Х - входной тензор, который и был представлен на рисунке в тетради
Y = data[inp_chars:]  # предсказание следующего символа (требуемые выходы, т.е для которых нужно построить прогноз) 

print(data.shape)

(22452, 34)


In [25]:
model = Sequential()
model.add(Input((inp_chars, # 2 аргумента, т.к размер батча автоматически добавится
                 num_characters)))  # при тренировке в рекуррентные модели keras подается сразу вся последовательность, поэтому в input теперь два числа. 1-длина последовательности, 2-размер OHE
model.add(SimpleRNN(128, activation='tanh'))  # рекуррентный слой на 500 нейронов
model.add(Dense(num_characters, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, 128)               20864     
                                                                 
 dense_4 (Dense)             (None, 34)                4386      
                                                                 
Total params: 25,250
Trainable params: 25,250
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model.fit(X, Y, batch_size=32, epochs=300)

Epoch 1/300
702/702 [==============================] - 2s 2ms/step - loss: 2.8598 - accuracy: 0.1987
Epoch 2/300
702/702 [==============================] - 1s 2ms/step - loss: 2.5905 - accuracy: 0.2517
Epoch 3/300
702/702 [==============================] - 1s 2ms/step - loss: 2.5091 - accuracy: 0.2640
Epoch 4/300
702/702 [==============================] - 2s 2ms/step - loss: 2.4706 - accuracy: 0.2732
Epoch 5/300
702/702 [==============================] - 2s 2ms/step - loss: 2.4480 - accuracy: 0.2754
Epoch 6/300
702/702 [==============================] - 2s 3ms/step - loss: 2.4301 - accuracy: 0.2787
Epoch 7/300
702/702 [==============================] - 2s 3ms/step - loss: 2.4132 - accuracy: 0.2836
Epoch 8/300
702/702 [==============================] - 2s 3ms/step - loss: 2.4008 - accuracy: 0.2859
Epoch 9/300
702/702 [==============================] - 2s 2ms/step - loss: 2.3838 - accuracy: 0.2955
Epoch 10/300
702/702 [==============================] - 2s 2ms/step - loss: 2.3676 - accura

In [11]:
# Функция для построения фразы на основе прогнозных значений
def buildPhrase(inp_str, str_len=50): # inp_str - начальные символы
    for i in range(str_len):
        x = []
        for j in range(i, i + inp_chars):
            x.append(tokenizer.texts_to_matrix(inp_str[j]))  # преобразуем символы в One-Hot-encoding

        x = np.array(x)
        inp = x.reshape(1, inp_chars, num_characters) # формирует начальные символы, по которым будется делать прогноз

        pred = model.predict(inp)  # предсказываем OHE четвертого символа
        d = tokenizer.index_word[pred.argmax(axis=1)[0]]  # получаем ответ в символьном представлении (нужный = наибольшее значение) 

        inp_str += d  # дописываем строку

    return inp_str

In [12]:
res = buildPhrase("утренн")
print(res)

утренние мысли в понедельникесли вы сможете сделать пози


In [31]:
res = buildPhrase("семей")
print(res)

семейной хуй знаете как вы огдемее слканел в жепстот де
